In [1]:
import os
import sys
import keras
import numpy as np
import tensorflow as tf
from keras import datasets
import matplotlib.pyplot as plt

sys.path.append(os.getcwd() + "/../")

from bfcnn import BFCNN, collage

In [2]:
# setup environment
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
# get dataset 
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.astype(np.float32)
x_train = np.expand_dims(x_train, axis=3)
x_test = x_test.astype(np.float32)
x_test = np.expand_dims(x_test, axis=3)
x_train = 255.0 - x_train
x_test = 255.0 - x_test

In [4]:
EPOCHS = 20
FILTERS = 32
NO_LAYERS = 5
MIN_STD = 1.0
MAX_STD = 100.0
LR_DECAY = 0.95
LR_INITIAL = 0.1
BATCH_SIZE = 64
CLIP_NORMAL = 1.0
INPUT_SHAPE = (28, 28, 1)
PRINT_EVERY_N_BATCHES = 1000

In [5]:
# build model
model = \
    BFCNN(
        input_dims=INPUT_SHAPE,
        no_layers=NO_LAYERS,
        filters=FILTERS,
        kernel_regularizer=keras.regularizers.l1(0.01))

In [ ]:
# train dataset
trained_model, history = \
    BFCNN.train(
        model=model, 
        input_dims=INPUT_SHAPE,
        dataset=x_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        min_noise_std=MIN_STD,
        max_noise_std=MAX_STD,
        lr_initial=LR_INITIAL,
        lr_decay=LR_DECAY,
        print_every_n_batches=PRINT_EVERY_N_BATCHES)


2021-03-30 12:41:09,399 INFO custom_callbacks.py:__init__:57] deleting existing training image in ./training/images
2021-03-30 12:41:09,401 INFO model.py:train:312] begin training


Epoch 1/20


/home/arxwn/Repositories/blind_image_denoising/venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


223/938 [======>.......................] - ETA: 17s - batch: 111.0000 - size: 64.0000 - loss: 27.8630 - mae_loss: 13.5818

In [ ]:
# summarize history for loss
plt.figure(figsize=(15,5))
plt.plot(history.history["loss"],
         marker="o",
         color="red", 
         linewidth=3, 
         markersize=6)
plt.grid(True)
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train"], loc="upper right")
plt.show()

In [ ]:
from math import log10

# calculate mse for different std
sample_test = x_test[0:1024,:,:,:]
sample_test_mse = []
sample_train = x_train[0:1024,:,:,:]
sample_train_mse = []
sample_std = []

for std_int in range(0, int(MAX_STD), 5):
    std = float(std_int)
    #
    noisy_sample_test = sample_test + np.random.normal(0.0, std, sample_test.shape)
    noisy_sample_test = np.clip(noisy_sample_test, 0.0, 255.0)
    results_test = trained_model.model.predict(noisy_sample_test)
    mse_test = np.mean(np.power(sample_test - results_test, 2.0))
    sample_test_mse.append(mse_test)
    #
    noisy_sample_train = sample_train + np.random.normal(0.0, std, sample_train.shape)
    noisy_sample_train = np.clip(noisy_sample_train, 0.0, 255.0)
    results_train = trained_model.model.predict(noisy_sample_train)
    mse_train = np.mean(np.power(sample_train - results_train, 2.0))
    sample_train_mse.append(mse_train)
    #
    sample_std.append(std)

In [ ]:
# summarize history for loss
plt.figure(figsize=(16,8))
plt.plot(sample_std,
         [log10(m) for m in sample_test_mse],
         color="red",
         linewidth=2)
plt.plot(sample_std,
         [log10(m) for m in sample_train_mse],
         color="green", 
         linewidth=2)
plt.grid(True)
plt.title("Mean Square Error / std")
plt.ylabel("log10(MSE)")
plt.xlabel("Additive normal noise standard deviation")
plt.legend(["test", "train"], loc="lower right")
plt.show()

In [ ]:
# draw test samples, predictions and diff
sample = x_test[0:64,:,:,:]
noisy_sample = sample + np.random.normal(0.0, MAX_STD, sample.shape)
noisy_sample = np.clip(noisy_sample, 0.0, 255.0)
results = trained_model.model.predict(noisy_sample)
    
plt.figure(figsize=(14,14))
plt.subplot(2, 2, 1)
plt.imshow(collage(sample), cmap="gray_r") 
plt.subplot(2, 2, 2)
plt.imshow(collage(noisy_sample), cmap="gray_r") 
plt.subplot(2, 2, 3)
plt.imshow(collage(results), cmap="gray_r") 
plt.subplot(2, 2, 4)
plt.imshow(collage(np.abs(sample - results)), cmap="gray_r") 
plt.show() 

In [ ]:
trained_model.save("./model.h5")

In [ ]:
import keras

m = trained_model.model

def get_conv2d_weights(
        model: keras.Model):
    weights = []
    for layer in model.layers:
        layer_config = layer.get_config()
        layer_weights = layer.get_weights()
        if not "layers" in layer_config:
            continue
        for i,l in enumerate(layer_config["layers"]):
            if l["class_name"] == "Conv2D":
                for w in layer_weights[i]:
                    w_flat = w.flatten()
                    weights.append(w_flat)
    return np.concatenate(weights)

weights = get_conv2d_weights(m)

plt.figure(figsize=(14,5))
plt.grid(True)
plt.hist(x=weights, bins=500, range=(-0.1,+0.1), histtype="bar", log=True)
plt.show()